In [ ]:
import plac
import random
import warnings
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding

In [ ]:
# Set the model, model directory & training iterations

model = None
model_dir = "ner_model"
n_iter = 30

In [ ]:
# Entity label

LABEL = "MONEY"

In [ ]:
# Training sentences
sentence1 = "the lessee will pay the lessor for the premises herein described in the schedule a monthly rent of rs.8000/- ruppes eight thousand only." #this rental agreement made and executed at bangalore on this 1st day april two thousand eleven 01/04/2011 between prof. k. parthasarathy and mr. veerabrahmam bathini"
sentence2 = "tenant is the proprietor of the business and he has agreed to pay rent rs.4,000.00 (rupees four thousand only) for above said premises with the following terms and conditions"
sentence3 = "to pay as rental the sum of six thousand five hundred pesos ip 6.500.001 per month, due and payable in advance from the 20th day of every month."

In [ ]:
sentence1[99:108], sentence2[71:82], sentence3[63:70]

('rs.8000/-', 'rs.4,000.00', '6.500.0')

In [ ]:
# Let's first include only 1 sentence as training sample 
TRAIN_DATA = [
    (
        sentence1,
        {"entities": [(99, 108, LABEL)]},
    )
]

In [ ]:
optimizer.eps

1e-08

In [ ]:
# Training pipeline

def performTraining(model=None, new_model_name="money", output_dir=None, n_iter=30, training_data=None):
    """Set up the pipeline and entity recognizer, and train the new entity."""
    if training_data is None:
      print("No training data available")
    random.seed(0)
    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank("en")  # create blank Language class
        print("Created blank 'en' model")
    # Add entity recognizer to model if it's not in the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner)
    # otherwise, get it, so we can add labels to it
    else:
        ner = nlp.get_pipe("ner")

    ner.add_label(LABEL)  # add new entity label to entity recognizer
    # Adding extraneous labels shouldn't mess anything up
    ner.add_label("VEGETABLE")
    if model is None:
        optimizer = nlp.begin_training()
    else:
        optimizer = nlp.resume_training()
    move_names = list(ner.move_names)
    # get names of other pipes to disable them during training
    pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
    # only train NER
    with nlp.disable_pipes(*other_pipes), warnings.catch_warnings():
        # show warnings for misaligned entity spans once
        warnings.filterwarnings("once", category=UserWarning, module='spacy')

        sizes = compounding(1.0, 4.0, 1.001)
        # batch up the examples using spaCy's minibatch
        for itn in range(n_iter):
            random.shuffle(training_data)
            batches = minibatch(training_data, size=sizes)
            losses = {}
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(texts, annotations, sgd=optimizer, drop=0.35, losses=losses)
            print("Losses", losses)

    # test the trained model
    test_text = "This table cost me rs.5000/-"
    doc = nlp(test_text)
    print("Entities in '%s'" % test_text)
    for ent in doc.ents:
        print("Text: {} \n Entity label: {}".format(ent.text, ent.label_))

    # save model to output directory
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.meta["name"] = new_model_name  # rename model
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)

        # test the saved model
        print("Loading from", output_dir)
        nlp2 = spacy.load(output_dir)
        # Check the classes have loaded back consistently
        assert nlp2.get_pipe("ner").move_names == move_names
        doc2 = nlp2(test_text)
        for ent in doc2.ents:
            print(ent.label_, ent.text)

In [ ]:
# Let's call the training pipeline

if __name__ == "__main__":
    performTraining(model=None,new_model_name='money', output_dir=model_dir, n_iter=n_iter, training_data=TRAIN_DATA)

Created blank 'en' model
Losses {'ner': 21.599997997283936}
Losses {'ner': 20.848118484020233}
Losses {'ner': 19.84271776676178}
Losses {'ner': 18.425461411476135}
Losses {'ner': 16.71864342689514}
Losses {'ner': 15.93656075000763}
Losses {'ner': 11.4530930519104}
Losses {'ner': 8.617580309510231}
Losses {'ner': 5.762429267168045}
Losses {'ner': 2.6686986149288714}
Losses {'ner': 2.2013553264550865}
Losses {'ner': 1.9739588457741775}
Losses {'ner': 1.9223721206763003}
Losses {'ner': 1.9330309255601605}
Losses {'ner': 1.776848449835768}
Losses {'ner': 1.8334963074832302}
Losses {'ner': 1.5087327015342566}
Losses {'ner': 1.8755208179717897}
Losses {'ner': 1.794322657662974}
Losses {'ner': 1.1032019947516574}
Losses {'ner': 0.9559521437691956}
Losses {'ner': 1.3516114754866067}
Losses {'ner': 1.0330744386683364}
Losses {'ner': 0.7807410949519635}
Losses {'ner': 1.5702260074419598}
Losses {'ner': 0.5926932974791725}
Losses {'ner': 0.7051590615662917}
Losses {'ner': 0.5648890975020926}
Loss

The model couldn't identify money entity from the test sample.

Let's add 2 more samples to the training data & call training pipeline once again.

In [ ]:
TRAIN_DATA = [
    (
        sentence1,
        {"entities": [(99, 108, LABEL)]},
    ),
    (
        sentence2,
        {"entities": [(71, 82, LABEL)]},
    ),
    (
        sentence3,
        {"entities": [(63, 70, LABEL)]},
    ),

]

In [ ]:
# Let's call the training pipeline

if __name__ == "__main__":
    performTraining(model=None,new_model_name='money', output_dir=model_dir, n_iter=n_iter,training_data=TRAIN_DATA)

Created blank 'en' model
Losses {'ner': 73.32691694796085}
Losses {'ner': 61.47118382900953}
Losses {'ner': 35.03948091343045}
Losses {'ner': 8.041758655977901}
Losses {'ner': 3.987580961374519}
Losses {'ner': 4.017226890122106}
Losses {'ner': 3.993789373705496}
Losses {'ner': 3.9941763617368466}
Losses {'ner': 3.9287818586906766}
Losses {'ner': 2.786556280392725}
Losses {'ner': 2.527673382473438}
Losses {'ner': 19.98103150883226}
Losses {'ner': 3.061448617907526}
Losses {'ner': 3.6678592957408505}
Losses {'ner': 1.7260034068903338}
Losses {'ner': 2.3383967845989346}
Losses {'ner': 2.3364651348334133}
Losses {'ner': 2.4578095383950336}
Losses {'ner': 0.9475172279891622}
Losses {'ner': 0.4963269995377414}
Losses {'ner': 0.0222017013084767}
Losses {'ner': 0.02028355797591746}
Losses {'ner': 0.2358366756755483}
Losses {'ner': 0.0016751272875501285}
Losses {'ner': 0.00035194331929905656}
Losses {'ner': 1.2418440735979232e-05}
Losses {'ner': 0.0013183230154974071}
Losses {'ner': 7.498649382